### 固有値問題

In [2]:
:dep nalgebra

In [8]:
extern crate nalgebra as na;

use na::{DMatrix, SymmetricEigen};

let a = DMatrix::from_row_slice(3,3, &[
    1.0, 0.5, 0.5,
    0.5, 1.0, 0.5,
    0.5, 0.5, 1.0,
]);

let eigen = SymmetricEigen::new(a);

println!("Eigenvalues: {:?}", eigen.eigenvalues);
println!("Eigenvectors: {:?}", eigen.eigenvectors);

Eigenvalues: VecStorage { data: [2.000000000000001, 0.5000000000000002, 0.5], nrows: Dyn(3), ncols: Const }
Eigenvectors: VecStorage { data: [0.5773502691896257, 0.5773502691896257, 0.5773502691896258, -0.816496580927726, 0.40824829046386296, 0.4082482904638631, 0.0, 0.7071067811865477, -0.7071067811865475], nrows: Dyn(3), ncols: Dyn(3) }


### 勾配法

In [2]:
// 1次元勾配法(Gradient Descent in 1D)

{
    struct Search {
        func: Box<dyn Fn(f64) -> f64>,
    }

    impl Search {
        fn new(func: Box<dyn Fn(f64) -> f64>) -> Self {
            Self { func }
        }

        fn diff_func(&self, x: f64) -> f64 {
            let h = 1e-6;
            ((self.func)(x + h) - (self.func)(x - h)) / (2.0 * h)
        }

        fn search_max(&self, t0: f64, h0: f64, epsilon: f64) -> f64 {
            let mut t = t0;
            let mut h = h0;

            while self.diff_func(t).abs() > epsilon {
                h = self.diff_func(t).signum() * h.abs();
                t = t + h;
                if (self.func)(t) > (self.func)(t - h) {
                    while (self.func)(t) >= (self.func)(t - h) {
                        h = 2.0 * h;
                        t = t + h;
                    }
                    t = t - h;
                    h = h / 2.0;
                } else {
                    while (self.func)(t) <= (self.func)(t - h) {
                        h = h / 2.0;
                        t = t - h;
                    }
                    t = t + h;
                    h = h / 2.0;
                }
            }
            t
        }
    }

    let f_grad_1d = Box::new(|x: f64| - 2.0 * (x - 2.0).powi(2));
    let search = Search::new(f_grad_1d);
    let x_max_1d = search.search_max(0.0, 0.01, 1e-6);
    println!("Maximum of f(x) = (x - 2)^2 is at x = {}", x_max_1d);
}

Maximum of f(x) = (x - 2)^2 is at x = 2.0000001525878894


()